<a href="https://colab.research.google.com/github/leoisqualified/data-analysis-and-ml/blob/main/Polysemy_Resolution_in_finance_statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline,BertModel
from bs4 import BeautifulSoup
import torch
import numpy as np
import pandas as pd
import spacy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [ ]:
import requests
from bs4 import BeautifulSoup

# URLs of the website to scrape
url = "https://www.cnbc.com/finance/"


response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

headlines = soup.find_all("a", attrs={'class':'Card-title'})

for headline in headlines:
    print(headline.text)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    headlines = soup.find_all('a', class_='LatestNews-headline')

    # Print the headlines
    for headline in headlines:
        print(headline.get_text())

else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Federal Reserve says all 31 banks in annual stress test withstood a severe hypothetical downturn
Stocks making biggest moves midday: Rivian Automotive, FedEx, Whirlpool and more
Morgan Stanley advisors about to get an OpenAI-powered assistant for grunt work
Stocks making the biggest moves premarket: Rivian, Nvidia, Micron, FedEx
China's EV architect says investing in Europe is a way forward
Your 401(k) is higher, but a new report says Americans need to save even more 
Europe is at risk of over-restricting AI and falling behind U.S. and China, Dutch prince says  
China's yuan internationalization needs more 'applications,' HKEX CEO
Stocks making the biggest moves midday: SolarEdge, Pool Corp., Carnival and more
Fewer Americans are buying life insurance. Here's when you might need it
Cisco is 'very optimistic' about its growing business with China EVs
Stocks making the biggest moves midday: ResMed, Nvidia, Anheuser-Busch and more
Stocks making the biggest moves before the bell: Nvidia, F

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
# model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# headlines = ['We are hoping to break-even in the next quarter',
#              'The investment firm is bullish on the prospects of renewable energy',
#              'We faced a bear market which impacted stocks',]

# for headline in headlines:
#   inputs = tokenizer(headline, return_tensors="pt")
#   outputs = model(**inputs)
#   probs = torch.nn.functional.softmax(outputs.logits, dim = -1)
#   labels = ['positive','neutral','negative']
#   sentiment = labels[torch.argmax(probs)]
#   print(f'Sentiment:{sentiment}')

In [ ]:
# Load FinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

sentence = "We faced a bear market which impacted stocks"
target_word = "bear"

# Tokenize and get the embeddings
inputs = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
outputs = model(**inputs)

# Get the token embeddings from the last hidden state
token_embeddings = outputs.last_hidden_state[0]

# Get the token IDs and find the index of the target word
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
target_idx = tokens.index(target_word)

# Extract the embedding for the target word
target_embedding = token_embeddings[target_idx].detach().numpy()

# Define a set of financial terms (this can be expanded)
financial_terms = ["bull", "bear", "market", "stocks", "investment", "equity", "shares", "debt", "bond", "loss"]

# Tokenize and get the embeddings for financial terms
financial_embeddings = {}
for term in financial_terms:
    term_inputs = tokenizer(term, return_tensors="pt", add_special_tokens=False)
    term_outputs = model(**term_inputs)
    term_embedding = term_outputs.last_hidden_state[0][0].detach().numpy()
    financial_embeddings[term] = term_embedding

# Compute cosine similarity between the target word and financial terms
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarities = {}
for term, embedding in financial_embeddings.items():
    similarity = cosine_similarity(target_embedding, embedding)
    similarities[term] = similarity

# Sort terms by similarity
sorted_terms = sorted(similarities.items(), key=lambda item: item[1], reverse=True)

# Print the top similar terms
print(f"Top words similar to '{target_word}' in the financial context:")
for term, similarity in sorted_terms:
    print(f"{term}: {similarity:.4f}")


Top words similar to 'bear' in the financial context:
bear: 0.2275
market: 0.1543
loss: 0.1497
bull: 0.1378
bond: 0.1166
investment: 0.0545
debt: 0.0417
stocks: 0.0294
shares: 0.0171
equity: 0.0146


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import spacy
import numpy as np

# Load FinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

# Load spaCy for POS tagging
nlp = spacy.load("en_core_web_sm")

# Sentence to analyze
sentence = "We faced a bear market which impacted stocks."

# Define a set of financial terms for each noun
financial_terms_bear_market = [
    "downturn", "recession", "decline", "slump", "bearish trend", "negative market",
    "market dip", "market crash", "economic downturn", "depressed market"
]

financial_terms_stocks = [
    "equities", "shares", "securities", "holdings", "equity securities", "common stock",
    "preferred stock", "corporate stock", "public stock", "trading shares"
]

financial_terms_market = [
    "exchange", "bourse", "trading floor", "marketplace", "securities market",
    "stock market", "equity market", "bond market", "commodities market",
    "forex", "financial market", "capital market", "derivatives market",
    "futures market", "options market", "OTC market", "primary market", "secondary market"
]

financial_terms = {
    "bear": financial_terms_bear_market,
    "market": financial_terms_market,
    "stocks": financial_terms_stocks
}

# Perform POS tagging to identify potential target words
doc = nlp(sentence)
target_words = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]

# Tokenize and get the embeddings for the sentence
inputs = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
outputs = model(**inputs)
token_embeddings = outputs.last_hidden_state[0]
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Function to compute cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Tokenize and get the embeddings for financial terms
financial_embeddings = {}
for key, term_list in financial_terms.items():
    for term in term_list:
        term_inputs = tokenizer(term, return_tensors="pt", add_special_tokens=False)
        term_outputs = model(**term_inputs)
        term_embedding = term_outputs.last_hidden_state[0][0].detach().numpy()
        financial_embeddings[term] = term_embedding

# Process each target word
for target_word in target_words:
    if target_word in tokens or f"{tokens[tokens.index(target_word)-1]} {target_word}" in financial_terms:
        target_idx = tokens.index(target_word)
        target_embedding = token_embeddings[target_idx].detach().numpy()

        # Determine which financial terms list to use
        if target_word in financial_terms:
            term_list = financial_terms[target_word]
        else:
            term_list = financial_terms.get(f"{tokens[tokens.index(target_word)-1]} {target_word}", [])

        # Compute similarities
        similarities = {}
        for term in term_list:
            embedding = financial_embeddings[term]
            similarity = cosine_similarity(target_embedding, embedding)
            similarities[term] = similarity

        # Sort terms by similarity
        sorted_terms = sorted(similarities.items(), key=lambda item: item[1], reverse=True)

        # Print the top similar terms
        print(f"Top words similar to '{target_word}' in the financial context:")
        for term, similarity in sorted_terms:
            print(f"{term}: {similarity:.4f}")
        print("\n")


Top words similar to 'bear' in the financial context:
market dip: 0.2909
negative market: 0.2841
bearish trend: 0.2632
market crash: 0.2140
recession: 0.1942
downturn: 0.1885
depressed market: 0.1780
slump: 0.1703
economic downturn: 0.1442
decline: 0.0912


Top words similar to 'market' in the financial context:
secondary market: 0.2586
options market: 0.2476
commodities market: 0.2303
bourse: 0.2124
forex: 0.2093
trading floor: 0.2073
financial market: 0.2065
capital market: 0.1989
OTC market: 0.1956
derivatives market: 0.1933
equity market: 0.1857
primary market: 0.1802
securities market: 0.1690
marketplace: 0.1680
bond market: 0.1552
stock market: 0.1521
exchange: 0.1511
futures market: 0.1175


Top words similar to 'stocks' in the financial context:
equity securities: 0.2246
corporate stock: 0.2130
securities: 0.2098
common stock: 0.1857
public stock: 0.1756
shares: 0.1576
preferred stock: 0.1460
trading shares: 0.1357
equities: 0.1349
holdings: 0.1222




In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load dataset
data = pd.read_csv('your_dataset.csv')

# Download and load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Preprocess the data
def preprocess_data(data, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []

    for headline in data:
        encoded_dict = tokenizer.encode_plus(
            headline,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Preprocess the headlines
input_ids, attention_masks = preprocess_data(data['final_headline'], tokenizer)

# Convert labels to torch tensors
labels = torch.tensor(data['label'].values)

# Extract BERT embeddings
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    cls_embeddings = outputs[0][:, 0, :].numpy()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(cls_embeddings, data['label'].values, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(cls_embeddings.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load dataset
data = pd.read_csv('your_dataset.csv')

# Step 1: Text cleaning
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

data['cleaned_headline'] = data['headline'].apply(clean_text)

# Step 2: Tokenization
data['tokenized_headline'] = data['cleaned_headline'].apply(word_tokenize)

# Step 3: Removing stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

data['headline_no_stopwords'] = data['tokenized_headline'].apply(remove_stopwords)

# Step 4: Lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

data['lemmatized_headline'] = data['headline_no_stopwords'].apply(lemmatize_tokens)

# Step 5: Feature extraction (TF-IDF)
data['final_headline'] = data['lemmatized_headline'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['final_headline'])

# Convert the labels to a numpy array
y = data['label'].values

# Display the shapes of the features and labels
print(X.shape)
print(y.shape)
